In [1]:
args = {
    "dataset_folder": "./train_10M",
    "epochs": 5
}

In [2]:
import os
from datasets import load_dataset
import datasets
import torch

In [3]:
!mkdir curricula

mkdir: cannot create directory ‘curricula’: File exists


### Curriculum 2023 dataset

The exact data from Thoma et al 2023. Note that the model does not see all examples in each epoch.

In [4]:
from datasets import load_dataset
import os
os.environ['HF_HOME'] = '/data/loriss21dm/cache'
# dev = load_dataset("babylm-anon/dev-data")

In [5]:
LAST_GEN_DIR = "last_gen"
EPOCHS_PER_STAGE = 10
datasets_stages = []
orders_stages = []

torch.manual_seed(0)
for folder in (sorted(os.listdir(LAST_GEN_DIR))):
    d = load_dataset("text", data_dir =os.path.join(LAST_GEN_DIR, folder), cache_dir='/data/loriss21dm/cache')["train"] 
    d = d.shuffle() # we shuffle within the stage
    d = datasets.Dataset.from_dict(d[0:len(d)//10]) # TODO
    datasets_stages.append(d)
    offset = orders_stages[-1][-1]+1 if len(orders_stages) else 0
    for _ in range(0,EPOCHS_PER_STAGE):
        orders_stages.append(torch.arange(offset,len(d)+offset))
    


dataset = datasets.concatenate_datasets(datasets_stages)

dataset.save_to_disk("./curricula/datasets/curriculum_100M_2023")
#assert torch.equal(torch.cat([o.flatten() for o in orders_stages]),torch.arange(0,len(dataset)))

torch.save(orders_stages, "./curricula/curriculum_100M_2023")
len(orders_stages)

Saving the dataset (0/1 shards):   0%|          | 0/1058753 [00:00<?, ? examples/s]

40

In [6]:
dataset

Dataset({
    features: ['text'],
    num_rows: 1058753
})

In [7]:
sum((len(o) for o in orders_stages)) / 10587551 


0.99999801653848

In [8]:
dataset

Dataset({
    features: ['text'],
    num_rows: 1058753
})

In [9]:
torch.tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
         18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35]).repeat(1,10,1),

(tensor([[[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
           17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
           34, 35],
          [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
           17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
           34, 35],
          [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
           17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
           34, 35],
          [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
           17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
           34, 35],
          [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
           17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
           34, 35],
          [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
           17, 18, 19, 20, 21, 

### Curriculum with 10M 2024 dataset

For training with the strategy in Thoma et al 2023. Note that the model does not see all examples in each epoch.

In [10]:
args["dataset_folder"] = "./train_10M"
curriculum = {
    "C1": ["childes.train", "open_subtitles.train"],
    "C2": ["switchboard.train", "bnc_spoken.train"],
    "C3": ["gutenberg.train", "simple_wiki.train"]
}

In [11]:
datasets_stages = []
orders_stages = []

torch.manual_seed(0)
for stage, files in curriculum.items():
    d = datasets.concatenate_datasets(
            [
                load_dataset("text", data_files =os.path.join(args["dataset_folder"], file))["train"] 
                for file in files
            ]
        )
    d = d.shuffle() # we shuffle with the stage
    datasets_stages.append(d)
    offset = orders_stages[-1][-1]+1 if len(orders_stages) else 0
    for i in range(0,5):
        orders_stages.append(torch.arange(offset,len(d)+offset))
    


dataset = datasets.concatenate_datasets(datasets_stages)
dataset.save_to_disk("./curricula/datasets/curriculum_10M_2024")
assert torch.equal(torch.cat([o.flatten() for o in orders_stages]),torch.arange(0,len(dataset)))

torch.save(orders_stages, "./curricula/curriculum_10M_2024")
orders_stages
dataset = dataset.shuffle()
dataset = datasets.Dataset.from_dict(dataset[0:len(dataset)//100]) # TODO
dataset.save_to_disk("./curricula/datasets/curriculum_10M_2024_eval")

Saving the dataset (0/1 shards):   0%|          | 0/1179014 [00:00<?, ? examples/s]

AssertionError: 

### Random Shuffle

This mimicks the default behaviour of Huggingface's `Trainer`: Randomly shuffle dataset after each epoch.
All data is shown at each epoch.

In [12]:
from datasets import load_dataset
import datasets
dataset = datasets.load_from_disk("./curricula/datasets/curriculum_10M_2024") # reuse one created above

In [13]:
assert torch.equal(torch.sort(torch.randperm(len(dataset))).values, torch.arange(0,len(dataset)))

In [14]:
args["epochs"] = 10
import torch 
torch.manual_seed(0)
random_order = torch.stack([torch.randperm(len(dataset)) for _ in range(0,args["epochs"])])
torch.save(random_order, "./curricula/curriculum_10M_2024_random")
random_order.shape

torch.Size([10, 1179014])

In [15]:
len(random_order[1])

1179014

### Curriculum Convolution with lognorm 

## Test random

In [16]:
from datasets import load_dataset
import torch 
torch.manual_seed(0)

#dataset = load_dataset("text", data_dir ="/data/loriss21dm/babylm/train_test", cache_dir='/data/loriss21dm/cache')["train"] 
dataset = datasets.load_from_disk("./curricula/datasets/curriculum_10M_2024") # reuse one created above
dataset = dataset.shuffle()
dataset = datasets.Dataset.from_dict(dataset[0:len(dataset)//1000]) # TODO
dataset.save_to_disk("./curricula/datasets/test")
args["epochs"] = 10
random_order = torch.stack([torch.randperm(len(dataset)) for _ in range(0,args["epochs"])])
torch.save(random_order, "./curricula/test_random_curriculum")


dataset = datasets.Dataset.from_dict(dataset[0:len(dataset)//10]) # TODO
dataset.save_to_disk("./curricula/datasets/test_eval")

random_order

Saving the dataset (0/1 shards):   0%|          | 0/1179 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/117 [00:00<?, ? examples/s]

tensor([[ 251,   62,  344,  ...,   21,  353,  118],
        [ 358,  931,  829,  ...,  341,  954,  115],
        [ 662,  283,  104,  ...,  893, 1049,  748],
        ...,
        [ 635,  621, 1080,  ...,  788,  130,  181],
        [ 505,  245, 1088,  ...,  900, 1119,   94],
        [ 757,  110,   69,  ...,  132, 1100,  623]])

In [17]:
max(random_order.flatten())

tensor(1178)

In [18]:
dataset

Dataset({
    features: ['text'],
    num_rows: 117
})